In [20]:
import statsmodels.api as sm
import statsmodels.stats.sandwich_covariance as sw
import numpy as np
import pandas as pd

In [24]:
OK = pd.read_excel(
    "data/OK.xlsx",
    sheet_name=0,
    engine="openpyxl"
)

OK.head()

,Date,Close
0,2020-01-02,54.99
1,2020-01-03,54.69
2,2020-01-06,54.67
3,2020-01-07,54.25
4,2020-01-08,54.35


In [27]:
SP500 = pd.read_excel(
    "data/SP500.xlsx",
    sheet_name=0,
    engine="openpyxl"
)

SP500.head()

,Date,Close
0,2020-01-02,3257.85
1,2020-01-03,3234.85
2,2020-01-06,3246.28
3,2020-01-07,3237.18
4,2020-01-08,3253.05


In [26]:
# r(t+1) = ln[P(t+1) / P(t)]
OK["log_return"] = np.log(OK["Close"] / OK["Close"].shift(1))

# r(t+1) = [P(t+1) - P(t)] / P(t)
OK["arith_return"] = OK["Close"].pct_change()     # pct_change() does exactly this

OK.head()

,Date,Close,log_return,arith_return
0,2020-01-02,54.99,NaN,NaN
1,2020-01-03,54.69,-0.005470,-0.005456
2,2020-01-06,54.67,-0.000366,-0.000366
3,2020-01-07,54.25,-0.007712,-0.007682
4,2020-01-08,54.35,0.001842,0.001843


In [13]:
#--y = np.asarray(OK['Close']).astype(float)
X = pd.to_datetime(OK['Date']).apply(lambda x: x.toordinal()) 
X = np.asarray(sm.add_constant(X))

In [14]:
model = sm.OLS(OK['Close'], X).fit()

In [15]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Close   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     1217.
Date:                Mon, 16 Jun 2025   Prob (F-statistic):          1.83e-159
Time:                        22:13:31   Log-Likelihood:                -2053.0
No. Observations:                 756   AIC:                             4110.
Df Residuals:                     754   BIC:                             4119.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.079e+04    311.054    -34.704      0.0

In [16]:
hac_cov = sw.cov_hac(model, 1)

In [17]:
s = model.get_robustcov_results(cov_type='HAC', maxlags=1)
print(s.summary())

                            OLS Regression Results                            
Dep. Variable:                  Close   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     333.7
Date:                Mon, 16 Jun 2025   Prob (F-statistic):           5.26e-62
Time:                        22:13:42   Log-Likelihood:                -2053.0
No. Observations:                 756   AIC:                             4110.
Df Residuals:                     754   BIC:                             4119.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.079e+04    594.031    -18.172      0.0